In [96]:
#  Import statements for start
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2
import seaborn as sns
import re
from typing import List, Dict

In [94]:
# Connect to PostgreSQL database to access dataset

# Connection details
dbname = "cse214"
user = "sahil"
host="localhost"
port="5432"

# Initialize connection
conn = psycopg2.connect(
        dbname=dbname,
        user=user,
        host=host,
        port=port
    )
print("Connection Successful")

Connection Successful


In [143]:
class SQLParser:
    def __init__(self, query, connection):
        self.query = query
        self.tables = []
        self.predicates = []
        self.joins = []
        self.columns = []

    def parse(self):
        from_pattern = r"FROM\s+(\w+)\s+\w+"
        join_pattern = r"JOIN\s+(\w+)\s+\w+"
        self.tables.extend(re.findall(from_pattern, self.query))
        self.tables.extend(re.findall(join_pattern, self.query))

        join_condition_pattern = r"ON\s+([\w.]+)\s*=\s*([\w.]+)"
        join_conditions = re.findall(join_condition_pattern, self.query)
        self.joins.extend(join_conditions)
        self._extract_columns_from_joins(join_conditions)

        where_pattern = r"WHERE\s+(.+)"
        where_match = re.search(where_pattern, self.query)
        if where_match:
            predicates = where_match.group(1).split("AND")
            self.predicates.extend([pred.strip() for pred in predicates])
            self._extract_columns_from_predicates(predicates)

    def _extract_columns_from_joins(self, join_conditions):
        for left_col, right_col in join_conditions:
            left_column_name = self._extract_column_name(left_col)
            right_column_name = self._extract_column_name(right_col)
    
            if left_column_name not in self.columns:
                self.columns.append(left_column_name)
            if right_column_name not in self.columns:
                self.columns.append(right_column_name)
    
    def _extract_columns_from_predicates(self, predicates):
        column_pattern = r"([\w.]+)\s*[<>=!]+\s*[\w.]+"
        for predicate in predicates:
            match = re.search(column_pattern, predicate.strip())
            if match:
                column_name = self._extract_column_name(match.group(1))
                if column_name not in self.columns:
                    self.columns.append(column_name)
    
    def _extract_column_name(self, column: str) -> str:
        return column.split(".")[-1]


    def get_parsed_data(self) -> Dict[str, List]:
        return {
            "tables": self.tables,
            "joins": [" = ".join(join) for join in self.joins],
            "predicates": self.predicates,
            "columns": self.columns
        }

In [144]:
query = """
SELECT * 
FROM e_small e1 
JOIN edges e2 
ON e1.target = e2.source 
WHERE e1.source = 1000 AND e2.target < 500;
"""
parser = SQLParser(query, conn)
parse = parser.parse()
parsed_data = parser.get_parsed_data()

print("Tables:", parsed_data["tables"])
print("Joins:", parsed_data["joins"])
print("Predicates:", parsed_data["predicates"])
print("Columns:", parsed_data["columns"])

Tables: ['e_small', 'edges']
Joins: ['e1.target = e2.source']
Predicates: ['e1.source = 1000', 'e2.target < 500;']
Columns: ['target', 'source']


In [140]:
class cardinality_estimation:
    def __init__(self, parsed_query):
        self.parsed_query = parsed_query
    
    def cardinality(self):
        try:
            cursor = self.connection.cursor()

            query = f"""
            SELECT SUM(out_table.out_degree * in_table.in_degree) AS worst_case_cardinality
            FROM (
                SELECT {self.columns[0]} AS join_key, COUNT(*) AS out_degree
                FROM {self.tables[0]}
                GROUP BY {self.columns[0]}
            ) AS out_table
            JOIN (
                SELECT {self.columns[1]} AS join_key, COUNT(*) AS in_degree
                FROM {self.tables[1]}
                GROUP BY {self.columns[1]}
            ) AS in_table
            ON out_table.join_key = in_table.join_key;
            """

            cursor.execute(query)
            result = cursor.fetchone()[0]
            cursor.close()

            return result
        except Exception as e:
            print(f"Error calculating degree sequences: {e}")
            return None